In [42]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns

### Считываем данные из файла

In [43]:
df = pd.read_csv('car.data', header=None, names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class'], delimiter=',')
df.head(10)

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying      1728 non-null object
maint       1728 non-null object
doors       1728 non-null object
persons     1728 non-null object
lug_boot    1728 non-null object
safety      1728 non-null object
class       1728 non-null object
dtypes: object(7)
memory usage: 94.6+ KB


### Data set info
В данном дата сете определяют класс машины по нижеперечисленным параметрам
#### Attribute Information:

#### Class Values:

- unacc
- acc
- good
- vgood

#### Attributes:

buying: vhigh, high, med, low.<br>
maint: vhigh, high, med, low.<br>
doors: 2, 3, 4, 5more.<br>
persons: 2, 4, more.<br>
lug_boot: small, med, big.<br>
safety: low, med, high.<br>

class - целевая переменная

#### В описании дата сета указано, что пропущенных значений нет, поэтому заменять или удалять Nan, не нужно

## Даммирование переменных

In [45]:
data_dummies = pd.get_dummies(df.loc[:, :'safety'])
display(data_dummies.head(2))

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,doors_5more,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,0,0,0,1,0,0,0,1,1,0,...,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,0,1,1,0,...,0,1,0,0,0,0,1,0,0,1


In [46]:
features = data_dummies.loc[:, :'safety_med']
x = features.values
y = df['class'].values


### Разделение датасета на тестовую и трэйновую

In [47]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

# Обучение дерева

### Дерево со случайными параметрами

In [48]:
from sklearn.tree import DecisionTreeClassifier


tree=DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=5)
tree.fit(X_train, y_train)
print('Результат на обучающей выборке: ', tree.score(X_train,y_train))
print('Результат на тестовой выборке: ',tree.score(X_test, y_test))

Результат на обучающей выборке:  0.9280397022332506
Результат на тестовой выборке:  0.9383429672447013


### Поиск наилучшего дерева путем перебора комбинаций max_depth, criterion, min_samples_leaf

In [49]:
table = pd.DataFrame(columns=['max_depth', 'criterion', 'min_samples_leaf', 'train_result', 'test_result'])
for criterion in ['entropy', 'gini']:
    for max_depth in range(5, 100):
        for min_leaf in range(1, 20):
            tree = DecisionTreeClassifier(max_depth=max_depth, random_state=0, criterion=criterion, min_samples_leaf=min_leaf)
            tree.fit(X_train, y_train)
            train_result = tree.score(X_train, y_train)
            test_result = tree.score(X_test, y_test)
            table = table.append({'max_depth': max_depth,
                         'criterion': criterion, 
                         'min_samples_leaf': min_leaf,
                         'train_result': train_result,
                         'test_result': test_result}, ignore_index=True)

table

,max_depth,criterion,min_samples_leaf,train_result,test_result
0,5,entropy,1,0.851944,0.870906
1,5,entropy,2,0.851944,0.870906
2,5,entropy,3,0.851944,0.870906
3,5,entropy,4,0.851944,0.870906
4,5,entropy,5,0.851944,0.870906
...,...,...,...,...,...
3605,99,gini,15,0.916460,0.921002
3606,99,gini,16,0.916460,0.921002
3607,99,gini,17,0.909843,0.924855
3608,99,gini,18,0.901572,0.917148


In [52]:
index = np.argmax(np.array(table['test_result']))
table.iloc[index]

max_depth                 13
criterion               gini
min_samples_leaf           1
train_result               1
test_result         0.980732
Name: 1957, dtype: object

И так, параметры наилучшего дерева для тестовой выборки(точность - 98,1%):
- max_depth: 7
- criterion: entropy
- min_samples_leaf: 11

<br>Данное дерево идеально работает на тренировочной выборке, что говорит о перетренированности. Однако в то же время дерево имеет очень высокую точность и для тестовой выборки. Отсюда вывод: тестовая и тренировочная выборки очень похожи друг на друга

### Визуализация дерева

In [51]:
best_tree = DecisionTreeClassifier(max_depth=12, random_state=0, criterion='entropy', min_samples_leaf=1)
best_tree.fit(X_train, y_train)
from sklearn.tree import export_graphviz
import pydot


best_tree = DecisionTreeClassifier(max_depth=7, random_state=0, criterion='entropy', min_samples_leaf=11)
best_tree.fit(X_train, y_train)
export_graphviz(best_tree, out_file='car_tree.dot', rounded = True, proportion = False, precision = 2, filled = True)
(graph, ) = pydot.graph_from_dot_file('car_tree.dot')

graph.write_png('car_tree.png')